In [ ]:
import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
def conv_block_3d(in_dim, out_dim, act_fn):
    model = nn.Sequential(
        nn.Conv3d(in_dim, out_dim, kernel_size=3, padding=1),
        nn.BatchNorm3d(out_dim),
        act_fn
    )
    return model

def conv_block_trans_3d(in_dim, out_dim, act_fn=nn.LeakyReLU(0.2, inplace=True)):
    model = nn.Sequential(
        nn.ConvTranspose3d(in_dim, out_dim, kernel_size=3, padding = 1, stride=2, output_padding=1),
        nn.BatchNorm3d(out_dim),
        act_fn
    )
    return model

def conv_full_block3d(in_dim, out_dim = -1, act_fn=nn.LeakyReLU(0.2, inplace=True)):
    if out_dim == -1:
        out_dim = in_dim * 2
    model = nn.Sequential(
        conv_block_3d(in_dim, in_dim, act_fn),
        conv_block_3d(in_dim, out_dim, act_fn)
    )
    return model
    
def conv_up_full_block3d(in_dim, out_dim, act_fn=nn.LeakyReLU(0.2, inplace=True)):
    model = nn.Sequential(
        nn.Conv3d(in_dim, out_dim, kernel_size=3, padding = 1),
        nn.BatchNorm3d(out_dim),
        act_fn,
        nn.Conv3d(out_dim, out_dim, kernel_size=3, padding = 1),
        nn.BatchNorm3d(out_dim),
        act_fn
        
    )
    return model

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_dim=1, out_dim=1, num_of_start_filters=32):
        super(UNet, self).__init__()
        act_fn = nn.LeakyReLU(0.2, inplace=True)

        print("\n------Initiating U-Net------\n")
        self.conv1 = nn.Conv3d(in_dim, num_of_start_filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(num_of_start_filters, num_of_start_filters * 2, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
                
        self.down2 = conv_full_block3d(num_of_start_filters * 2)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        
        self.down3 = conv_full_block3d(num_of_start_filters * 4)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        
        self.bridge = conv_full_block3d(num_of_start_filters * 8)
        
        self.trans1 = conv_block_trans_3d(num_of_start_filters * 16, num_of_start_filters * 16)
        
        
        self.up1 = conv_up_full_block3d(768, 256)
        self.trans2 = conv_block_trans_3d(256, 256)
        
        self.up2 = conv_up_full_block3d(384, 128)
        self.trans3 = conv_block_trans_3d(128, 128)
        
        self.up3 = conv_up_full_block3d(192, 64)
        
        self.out = nn.Conv3d(64, in_dim, kernel_size=1)
        
    def forward(self, x):
        x = self.conv1(x)
        conv2 = self.conv2(x)
        
        pool_1 = self.pool1(conv2)
        
        #print("after first down :")
        #print(pool_1.size())

        down2 = self.down2(pool_1)
        pool2 = self.pool2(down2)
        
        #print("after second down :")
        #print(pool2.size())
        
        down3 = self.down3(pool2)
        pool3 = self.pool3(down3)
        
       # print("after third down :")
       # print(pool3.size())
        
        bridge = self.bridge(pool3)
        
        #print("after bridge :")
        #print(bridge.size())
        
        trans1 = self.trans1(bridge)
        #print("after first transpose: ")
       # print(trans1.size())
        
        #print("concatenate maps along axis 1 of size :")
        #print(trans1.size())
       # print(down3.size())
        concat1 = torch.cat([trans1, down3], dim=1)
        
        up1 = self.up1(concat1)
        
        trans2 = self.trans2(up1)
        concat2 = torch.cat([trans2, down2], dim=1)
        
        up2 = self.up2(concat2)
        
        trans3 = self.trans3(up2)
        
        concat3 = torch.cat([trans3, conv2], dim=1)
        
        up3 = self.up3(concat3)
        
        #print("Size before 1x1x1 convolution:")
        #print(up3.size())
        #print("Size after 1x1x1 conv:")
        tmp = self.out(up3)
       # print(tmp.size())
        #out = nn.LeakyReLU(tmp, 0.2, inplace=True)
        
        
        return F.leaky_relu(tmp)
        
        
        
        

In [ ]:
image_size = 64

In [ ]:
def dice_loss(pred, target):
    eps = 0.0001
    pred = pred.contiguous()
    target = target.contiguous() 
    inter = torch.dot(pred.view(-1), target.view(-1))
    #print(pred.view(-1).size())
    union = torch.sum(pred) + torch.sum(target) + eps
    t = (2 * inter.float() + eps) / union.float()
    print("loss :" + str(1 - t))
    return 1 - t

# Checking funcs

In [ ]:
    import numpy as np
    data1 = Variable(torch.from_numpy(np.zeros((1, 1, 32, 64, 64))), requires_grad=True)
    data2 = Variable(torch.from_numpy(np.ones((1, 1, 32, 64, 64))), requires_grad=True)
    target = Variable(torch.from_numpy(np.zeros((1, 1, 32, 64, 64))))

    
    data = torch.add(data1, data2)
    
    data1.grad == None
    data

In [ ]:
loss = dice_loss(data, target)
#type(loss)

print(str(loss))
loss.backward()
loss.grad
data1.grad


# End of check

In [ ]:
import numpy as np
def train(args, model, device, pipeline, optimizer, epoch, loss_hist, batch_size = 1):
    model.train()
    
    # [batch_size, channels, z, x, y]
    data = np.zeros((batch_size, 1, 32, 64, 64))
    target = np.zeros((batch_size, 1, 32, 64, 64))
    
    for batch_idx in range(3):
        batch_crops = pipeline.next_batch(1)
        
        # create shape for batch 
        for s_num in range(batch_size):
            data[s_num][0] = (batch_crops.get(s_num, 'images'))
            target[s_num][0] = (batch_crops.get(s_num, 'masks'))
            
        data_t = Variable(torch.from_numpy(data), requires_grad=True)
        target_t = torch.from_numpy(target)
            
            
        data_t, target_t = data_t.to(device), target_t.to(device)
        print("data size input: ")
        print(data_t.size())
        
        optimizer.zero_grad()
        output = model(data_t.double())
        print("data size output: ")
        print(output.size())
        
        loss = dice_loss(output, target_t)
        loss.backward()
                
        optimizer.step()
                
        loss_hist.append(loss)
        print('Batch: ' + str(batch_idx) + ' Loss: ' + str(loss.item()))
        

In [ ]:
from radio.dataset import FilesIndex, Dataset, Pipeline
from radio import CTImagesMaskedBatch as CTIMB

index = FilesIndex(path='Data/subset0/*.mhd', no_ext=True) # dataset initialization
lunaset = Dataset(index=index, batch_class=CTIMB) # download to data to RAM

In [ ]:
import pandas as pd

lunaset.split([0.1, 0.9])  # 10 % goes for training

# load annotations from csv
nodules_df = pd.read_csv('Data/CSVFILES/annotations.csv')

pipeline = (Pipeline()
        .load(fmt='raw') # set .raw format
        .normalize_hu(-1000, 400)  # normalize pixel colors
        .fetch_nodules_info(nodules_df) # extract nodules info
        .unify_spacing(shape=(128, 256, 256), spacing=(2, 2, 2)) # set uniform scale for CT pics
        .create_mask() # extract nodule masks
        .sample_nodules(nodule_size=(32, 64, 64), batch_size=1) # crop nodules and generate batchers
       )

lunapipe = (lunaset.train >> pipeline)  # pass lunaset to pipeline

In [ ]:
print(len(lunaset.train))

In [ ]:
model = UNet().to('cpu')
model = model.double()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
params = list(model.parameters())



In [ ]:
loss_hist = []
train(None, model, 'cpu', lunapipe, optimizer, 1, loss_hist)